In [1]:
from flask import Flask, render_template, request, jsonify
from surprise.model_selection import KFold, GridSearchCV
from flaskext.mysql import MySQL
from surprise import dump, KNNBasic
from tqdm import tqdm
from operator import itemgetter
from scipy import stats

import pickle
import pymysql  # mysql error handling
import functools
import itertools
import json
import random

import numpy as np
import pandas as pd

In [2]:
from rec_utils import *
from db_helper import DbHelper

In [3]:
app = Flask(__name__)
app.config.from_pyfile('config.py')

mysql = MySQL()
mysql.init_app(app)

conn = mysql.connect()
cursor = conn.cursor()

In [4]:
db = DbHelper(conn, cursor)


## Registro

In [5]:
socio = {"apPaterno": "Mendoza", "apMaterno": "Hernandez", "nombre": "Alma", 
        "edad": 22, "genero": "F", "email": "almis@gmail.com", "passwd": "frio"}
db.register(socio)

{'email_used': False, 'success': True}

In [6]:
db.register(socio)

(1062, "Duplicate entry 'almis@gmail.com' for key 'socio.c_uniq_email_passwd'")


{'email_used': True, 'success': False}

In [7]:
socio = {"apPaterno": "Fragoso", "apMaterno": "Hernandez", "nombre": "Abigail", 
        "edad": 19, "genero": "F", "email": "almis@gmail.com", "passwd": "frio"}
db.register(socio)

(1062, "Duplicate entry 'almis@gmail.com' for key 'socio.c_uniq_email_passwd'")


{'email_used': True, 'success': False}

In [8]:
db.write('delete from socio where passwd = "frio"')

## Login

In [25]:
email, passwd = 'AZ0PCEZUTF5QO@gmail.com', 'passfd'
db.login(email, passwd)

{'ok': False}

In [17]:
email, passwd = 'almis@gmail.com', 'frio'
db.login(email, passwd)

In [10]:
passwd = 'incorrecto'
db.login(email, passwd)

{'email_found': False, 'correct_passwd': False, 'idSocio': False}

In [11]:
email = 'noexiste@gmail.com'
db.login(email, 'falsooo')

{'email_found': False, 'correct_passwd': False, 'idSocio': False}

## product info

In [12]:
raw_iid = 'B005FIWTHO'
db.get_product_info(raw_iid)

{'idProducto': 'B005FIWTHO',
 'nombre': 'Quicken Premier 2012',
 'marca': 'Intuit',
 'precioUnitario': 44.96298689479438,
 'idSubCat': 17}

In [13]:
raw_iids = ['B005FIWTHO','B0053WX3AY','B00EZPXYP4','B00BFNCFZ4']
db.get_products_info(raw_iids)

[{'idProducto': 'B0053WX3AY',
  'nombre': 'Dragon NaturallySpeaking Home, European Version 11.5',
  'marca': 'Nuance Communications, Inc.',
  'precioUnitario': 44.96298689479438,
  'idSubCat': 17},
 {'idProducto': 'B005FIWTHO',
  'nombre': 'Quicken Premier 2012',
  'marca': 'Intuit',
  'precioUnitario': 44.96298689479438,
  'idSubCat': 17},
 {'idProducto': 'B00BFNCFZ4',
  'nombre': 'VideoStudio Pro X6 [OLD VERSION]',
  'marca': 'Corel',
  'precioUnitario': 19.95,
  'idSubCat': 17},
 {'idProducto': 'B00EZPXYP4',
  'nombre': 'NORTON 360 21.0 EN 1U 3LIC MM',
  'marca': 'Symantec',
  'precioUnitario': 44.96298689479438,
  'idSubCat': 17}]

## Recomendaciones 

In [21]:
def get_recs(uid):
    """
    Returns:
        {was_possible: bool[, products: list<{idProducto, nombre, marca, precioUnitario, idSubCat}>]}: 
            was_possible = True Si fue posible generar una recomendacion   
            products: Lista de atributos de productos.
    """
    res = {'was_possible': False}
    try:
        iid_recs = get_top_item_based(algo, uid, trainset, sims)  # if raw_id not in trainset it raises error
        res['products'] = db.get_products_info(iid_recs)
        res['was_possible'] = True
    except ValueError as e:
        pass
    return res

In [5]:
# config to keep same experiments
my_seed = 0
random.seed(my_seed)
np.random.seed(my_seed)

In [6]:
algo, sims, trainset, testset = get_rec_sys_resources(fname='software_reviews_with_outliers.csv')

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.


In [28]:
uid = testset[0][0]
raw_iids = get_top_item_based(algo, uid, trainset, sims)  # if raw_id not in trainset it raises error

If there's no uid in trainset ValueError is raised.

In [8]:
not_in_trainset = []
for uid in tqdm(map(lambda x: x[0], testset), total=len(testset)):
    try:
        get_top_item_based(algo, uid, trainset, sims)
    except ValueError as e:
        not_in_trainset.append(uid)
len(not_in_trainset)

100%|██████████| 4378/4378 [00:46<00:00, 93.87it/s] 


11

### valoraciones reales

In [11]:
df = pd.read_csv('./model/software_reviews.csv')
uid_ratings = df[df.reviewerID == uid].asin.values
db.get_products_info(uid_ratings)

{'productsInfo': [{'idProducto': 'B000HCZ8EO',
   'nombre': 'Microsoft Office Home and Student 2007 [Old Version]',
   'marca': 'Microsoft',
   'precioUnitario': 149.99,
   'idSubCat': 17},
  {'idProducto': 'B004E9SKDM',
   'nombre': 'Microsoft Windows 7 Anytime Upgrade (Starter to Home Premium) [Online Code]',
   'marca': 'Microsoft',
   'precioUnitario': 44.96298689479438,
   'idSubCat': 17},
  {'idProducto': 'B008RA5A00',
   'nombre': 'Quicken Deluxe 2013',
   'marca': 'Intuit',
   'precioUnitario': 44.96298689479438,
   'idSubCat': 17},
  {'idProducto': 'B00BR082FW',
   'nombre': 'CCleaner Professional System Optimization Tool  Unlimited Home Use',
   'marca': 'Piriform',
   'precioUnitario': 44.96298689479438,
   'idSubCat': 17}]}

In [12]:
iid_recs = get_top_item_based(algo, uid, trainset, sims, k=5)  # if raw_id not in trainset it raises error
db.get_products_info(iid_recs)

{'productsInfo': [{'idProducto': 'B002SRNS7A',
   'nombre': 'Norton Ghost 15.0 - 1 PC',
   'marca': 'Symantec',
   'precioUnitario': 179.95,
   'idSubCat': 17},
  {'idProducto': 'B005FIWTHO',
   'nombre': 'Quicken Premier 2012',
   'marca': 'Intuit',
   'precioUnitario': 44.96298689479438,
   'idSubCat': 17},
  {'idProducto': 'B00M76N6MO',
   'nombre': 'Quicken For Mac Personal Finance &amp; Budgeting Software 2015 [Old Version]',
   'marca': 'Intuit',
   'precioUnitario': 44.96298689479438,
   'idSubCat': 17},
  {'idProducto': 'B00MUY6LL6',
   'nombre': 'Trend Micro Premium Security 2015 - 5 Devices [OLD VERSION][OLD VERSION]',
   'marca': 'Trend Micro',
   'precioUnitario': 44.96298689479438,
   'idSubCat': 17},
  {'idProducto': 'B00U7LCE6A',
   'nombre': 'CCleaner Free [Download]',
   'marca': 'Piriform',
   'precioUnitario': 44.96298689479438,
   'idSubCat': 17}]}

## outliers

In [13]:
# df = pd.read_csv('./model/software_reviews_with_outliers.csv')
print (list(map(len, (np.unique(df.reviewerID), np.unique(df.asin)))))
df.head()

[3176, 4494]


,overall,reviewerID,asin,reviewerName,unixReviewTime
0,4.0,A38NELQT98S4H8,0321719816,WB Halper,1287532800
1,4.0,A3QJU4FEN8PQSZ,0321719816,Grimmy,1287360000
2,5.0,ACJT8MUC0LRF0,0321719816,D. Fowler,1287187200
3,5.0,AYUF7YETYOLNX,0321719816,Bryan Newman,1286841600
4,5.0,A31ICLWQ9CSHRS,0321719816,Al Swanson,1286409600


In [14]:
# def get_outliers_df(df, group_on, num_col, fun='count', threshold=3.):
group_on = 'asin'
num_col = 'overall'
fun = 'count'
threshold = 3.

In [15]:
grp = df.groupby(group_on)[num_col]
grp = grp.count() if fun == 'count' else grp.sum()
z_vals = (grp - grp.mean()) / grp.std()
idx = abs(z_vals) > threshold  # idx.index: str, idx.values: bool
outlier_ids = idx[idx].index
    
grp.describe()

count    4494.000000
mean        4.870271
std         8.780852
min         1.000000
25%         1.000000
50%         2.000000
75%         4.000000
max       128.000000
Name: overall, dtype: float64

In [16]:
df_without_outliers = df[~df.asin.isin(outlier_ids)]
df_without_outliers.to_csv('./model/software_reviews_no_outliers.csv', index=False)
df.shape

(21887, 5)

In [17]:
df_without_outliers.shape

(17791, 5)

In [18]:
reader = Reader(rating_scale=(1, 5), )
cols = ['reviewerID', 'asin', 'overall']
data = Dataset.load_from_df(df[cols], reader)

In [19]:
sim_options = {'name': 'pearson', 'user_based': False}
algo = KNNBasic(sim_options=sim_options)
test_precision_recall(algo, data)

Computing the pearson similarity matrix...
Done computing similarity matrix.
precision
0.15658082975679544
recall
0.11771936099189315

Computing the pearson similarity matrix...
Done computing similarity matrix.
precision
0.14790863668807994
recall
0.10870925529383778

Computing the pearson similarity matrix...
Done computing similarity matrix.
precision
0.1485883030826851
recall
0.11173963364974604

Computing the pearson similarity matrix...
Done computing similarity matrix.
precision
0.15357951329937747
recall
0.11442467054733609

Computing the pearson similarity matrix...
Done computing similarity matrix.
precision
0.15021321961620468
recall
0.11112092598233325



In [20]:
reader = Reader(rating_scale=(1, 5), )
data = Dataset.load_from_df(df_without_outliers[cols], reader)
algo = KNNBasic(sim_options=sim_options)
test_precision_recall(algo, data)

Computing the pearson similarity matrix...
Done computing similarity matrix.
precision
0.06061343719571568
recall
0.043354430379746844

Computing the pearson similarity matrix...
Done computing similarity matrix.
precision
0.06803874092009685
recall
0.049585495215035164

Computing the pearson similarity matrix...
Done computing similarity matrix.
precision
0.06132075471698113
recall
0.04416553048628521

Computing the pearson similarity matrix...
Done computing similarity matrix.
precision
0.07823462228748573
recall
0.05454460696751445

Computing the pearson similarity matrix...
Done computing similarity matrix.
precision
0.06604696673189824
recall
0.046049995340601994



In [29]:
def get_real_ratings(df, uid):
    uid_ratings = df[df.reviewerID == uid].asin.values
    return db.get_products_info(uid_ratings)
    
algo, sims, trainset, testset = get_rec_sys_resources(fname='software_reviews_no_outliers.csv')   


Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.


In [30]:
uid = testset[0][0]
get_real_ratings(df_without_outliers, uid)

{'productsInfo': [{'idProducto': 'B000HKD8PM',
   'nombre': 'Acd Systems ACDSee Photo Editor',
   'marca': 'ACD Systems',
   'precioUnitario': 44.96298689479438,
   'idSubCat': 17},
  {'idProducto': 'B001EJVQDG',
   'nombre': 'Corel Paint Shop Pro Photo X2 Ultimate [Old Version]',
   'marca': 'Corel',
   'precioUnitario': 44.96298689479438,
   'idSubCat': 17},
  {'idProducto': 'B004Q0PT3I',
   'nombre': 'Windows 7 Home Premium SP1 64bit (OEM) System Builder DVD 1 Pack [Old Packaging]',
   'marca': 'Microsoft',
   'precioUnitario': 44.96298689479438,
   'idSubCat': 17},
  {'idProducto': 'B005VDPOR0',
   'nombre': 'Topaz Complete Plug-In Bundle for Photoshop (Mac &amp; Windows)',
   'marca': 'Topaz Labs',
   'precioUnitario': 44.96298689479438,
   'idSubCat': 17},
  {'idProducto': 'B00F2BDP9K',
   'nombre': 'Cyberlink PowerDirector 12 Ultimate',
   'marca': 'Cyberlink',
   'precioUnitario': 44.96298689479438,
   'idSubCat': 17},
  {'idProducto': 'B00NMPZ90Q',
   'nombre': 'Adobe Photosho

In [31]:
iid_recs = get_top_item_based(algo, uid, trainset, sims, k=5)  # if raw_id not in trainset it raises error
db.get_products_info(iid_recs)

{'productsInfo': [{'idProducto': 'B009SPL2K0',
   'nombre': 'Office Professional 2013 Key Card 1PC/1User',
   'marca': 'Microsoft',
   'precioUnitario': 44.96298689479438,
   'idSubCat': 17},
  {'idProducto': 'B00HRQB28Y',
   'nombre': 'Sony Movie Studio 13 Suite [OLD VERSION]',
   'marca': 'Sony',
   'precioUnitario': 29.98,
   'idSubCat': 17},
  {'idProducto': 'B00O66FYU0',
   'nombre': 'Adobe Creative Cloud Photography plan (Photoshop CC + Lightroom) Student and Teacher [Key Card] - Validation Required',
   'marca': 'Adobe',
   'precioUnitario': 44.96298689479438,
   'idSubCat': 17},
  {'idProducto': 'B015PIIF80',
   'nombre': 'MAGIX Movie Edit Pro 2016',
   'marca': 'MAGIX',
   'precioUnitario': 15.98,
   'idSubCat': 17},
  {'idProducto': 'B0169RGE7U',
   'nombre': 'Roxio Creator NXT 4 [Old Version]',
   'marca': 'Roxio',
   'precioUnitario': 29.99,
   'idSubCat': 17}]}

In [17]:
def f():
    
    data = Dataset.load_builtin('ml-100k')
    kf = KFold(n_splits=5)
    algo = KNNBasic(sim_options=sim_options)

    for trainset, testset in kf.split(data):
        algo.fit(trainset)
        predictions = algo.test(testset)
        precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)

        # Precision and recall can then be averaged over all users
        print(sum(prec for prec in precisions.values()) / len(precisions))
        print(sum(rec for rec in recalls.values()) / len(recalls))
        
        
f()

Computing the pearson similarity matrix...
Done computing similarity matrix.
0.3412420382165609
0.14531583120487446
Computing the pearson similarity matrix...
Done computing similarity matrix.
0.331210191082803
0.13600910708216818
Computing the pearson similarity matrix...
Done computing similarity matrix.
0.307714083510262
0.13415813126818335
Computing the pearson similarity matrix...
Done computing similarity matrix.
0.320222929936306
0.13534876744745641
Computing the pearson similarity matrix...
Done computing similarity matrix.
0.326910828025478
0.1352041211100263


## Frio

In [88]:
df = pd.read_csv('./model/software_reviews_no_outliers.csv')
good_ratings_df = df[df['overall'] >= 4]

In [87]:
def get_top_global(good_ratings_df, n=7):
    # select those good ratings. group them by iid, sum their values and sort them desc
    top_global_grp = good_ratings_df.groupby('asin')['overall'].sum().sort_values(ascending=False)
    total = len(top_global_grp)
    iids = np.random.choice(top_global_grp.index.values[:total // 10], n)  # from the 10% most rated, pick 10
    return db.get_products_info(iids)


get_top_global(good_ratings_df)

{'productsInfo': [{'idProducto': 'B0017I8NQM',
   'nombre': 'Wordperfect Office X4 Standard',
   'marca': 'Corel',
   'precioUnitario': 44.96298689479438,
   'idSubCat': 17},
  {'idProducto': 'B005N2CS8M',
   'nombre': 'Family Tree Maker Deluxe [OLD VERSION]',
   'marca': 'Nova Development US',
   'precioUnitario': 44.96298689479438,
   'idSubCat': 17},
  {'idProducto': 'B008S0IE5M',
   'nombre': 'Quicken Home &amp; Business 2013',
   'marca': 'Intuit',
   'precioUnitario': 44.96298689479438,
   'idSubCat': 17},
  {'idProducto': 'B0094NY3R0',
   'nombre': 'Windows 8 System Builder OEM DVD  64-Bit [Old Packaging]',
   'marca': 'Microsoft',
   'precioUnitario': 44.96298689479438,
   'idSubCat': 17},
  {'idProducto': 'B00MHZ71G2',
   'nombre': 'Norton Security with Backup (For 10 Devices) [Online Code]',
   'marca': 'Symantec',
   'precioUnitario': 44.96298689479438,
   'idSubCat': 17},
  {'idProducto': 'B00O66FYU0',
   'nombre': 'Adobe Creative Cloud Photography plan (Photoshop CC + Ligh

## Agregar productos comprados

In [22]:
req = {"idSocio": "1", "idProductos": [], "cantidades": []}
cursor.execute("SELECT idProducto FROM producto LIMIT 5")
iids = list(map(itemgetter(0), cursor.fetchall()))
req["idProductos"] = iids
req["cantidades"] = [1 for _ in iids]
req

{'idSocio': '1',
 'idProductos': ['1', '2', '3', '4', '5'],
 'cantidades': [1, 1, 1, 1, 1]}

In [ ]:
"""
{
	"idSocio": "1",
	"idProductos": ["1","2","3","4","5"],
	"cantidades": [1,1,1,1,1]
}
"""

In [23]:
db.insert_hist(req)

{'success': True}

In [24]:
db.read('select * from historial')

(('1', '1', datetime.datetime(2021, 1, 8, 17, 9, 56), 1),
 ('1', '2', datetime.datetime(2021, 1, 8, 17, 9, 56), 1),
 ('1', '3', datetime.datetime(2021, 1, 8, 17, 9, 56), 1),
 ('1', '4', datetime.datetime(2021, 1, 8, 17, 9, 56), 1),
 ('1', '5', datetime.datetime(2021, 1, 8, 17, 9, 56), 1))

In [25]:
db.write('delete from historial')

## Pendiente

In [26]:
res = {"idSocio": "1", "idProducto": "1"}
db.insert_pendiente(res)

{'success': True}

In [27]:
db.read('select * from pendiente')

(('1', '1'),)

In [28]:
db.write('delete from pendiente')